<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 목적

Sigmoid를 activation function으로 사용하는 깊은 신경망의 학습정체 원인은 vanishing gradient때문일까? 아니면 모델이 깊어짐에 따라 증가한 비선형성으로 optimizer가 local minimum에 빠졌기 때문일까? 이 프로젝트에서는 위의 의문을 해결하기위해 몇가지 가정을 세운후 간단한 이진분류모델로 실험해보고자한다.

# 가정1:학습정체의 원인이 vanishing gradient일 경우


실험1:

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

# data

In [ ]:
N=20
random0=torch.randn(int(N/2),1)
random5=torch.randn(int(N/2),1)+8

class1_data=torch.hstack([random0,random5])
class2_data=torch.hstack([random5,random0])

class1_label=torch.ones(int(N/2),1)
class2_label=torch.zeros(int(N/2),1)
class3_label=torch.ones(int(N/2),1)

X=torch.vstack([class1_data,class2_data])
y=torch.vstack([class1_label,class2_label])

plt.plot(class1_data[:,0],class1_data[:,1],'bo')
plt.plot(class2_data[:,0],class2_data[:,1],'ro')
# plt.plot(class3_data[:,0],class3_data[:,1],'bo')
plt.xlabel('x1')
plt.ylabel('x2')
plt.grid()

# 1

In [ ]:
class Sigmoid_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(nn.Linear(2, 10), nn.Sigmoid(),
                                   nn.Linear(10, 10), nn.Sigmoid(),
                                   nn.Linear(10, 10), nn.Sigmoid(),
                                   nn.Linear(10, 10), nn.Sigmoid(),
                                   nn.Linear(10, 10), nn.Sigmoid(),
                                   nn.Linear(10, 10), nn.Sigmoid(),
                                   nn.Linear(10, 1), nn.Sigmoid())

    def forward(self, x):
        x = self.layer(x)
        return x

In [ ]:
s_model = Sigmoid_model()
print(s_model.parameters)
print(s_model.layer[0].weight[0,0])

In [ ]:
a = s_model.layer[8].weight[3,4].tolist()
b = torch.linspace(a-50,a+50,10000)

In [ ]:
Loss = []

for i in b:
    s_model.layer[8].weight.requires_grad = False
    s_model.layer[8].weight[0,0] = i
    s_model.layer[8].weight.requires_grad = True
    loss = F.binary_cross_entropy(s_model(X[0]), y[0])
    Loss.append(round(loss.tolist(), 3))


In [ ]:
plt.plot(b, Loss)
plt.show()

In [ ]:
a = s_model.layer[10].weight[0,0].detach().numpy() # detach했지만 여전히 그라디언트 추적중

In [ ]:
print(s_model.layer[10].weight)

In [ ]:
s_model.layer[10].weight.requires_grad = False
s_model.layer[10].weight[0,0] = torch.tensor(3)

In [ ]:
s_model.layer[10].weight.requires_grad = True
print(s_model.layer[10].weight)